# Đồ án cuối kỳ

# Lập trình song song ứng dụng

# Training a Machine Learning model (Random Forest)



*   Link github: https://github.com/tangnhanqui/CSC14116



# Cài đặt thư viện cần thiết

In [ ]:
import numpy as np
import pandas as pd
import time
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from numba import jit, prange
from numba import config
import seaborn as sn
from sklearn import *
from sklearn.ensemble import *
from sklearn.datasets import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.neural_network import *
from sklearn.model_selection import GridSearchCV
config.THREADING_LAYER = 'omp'

In [ ]:
results = pd.DataFrame(columns=['time (s)'], dtype=float)
results # Bảng biểu lưu lại thời gian chạy của tất cả các phiên bản

,time (s)


#Tổng quan vấn đề
* Random forests là một mô hình có nhiều ứng dụng trong thực tế hiện nay, trong nhiều lĩnh vực khác nhau, chẳng hạn như ngân hàng, thị trường tài chính, y học, thương mại điện tử.
* Ví dụ, trong lĩnh vực ngân hàng, nó được sử dụng để phát hiện những khách hàng sẽ sử dụng dịch vụ ngân hàng thường xuyên hơn những người khác và trả nợ đúng hạn. Trong lĩnh vực này, nó cũng được sử dụng để phát hiện hành vi gian lận của khách hàng muốn gây hại cho ngân hàng.
* Trong lĩnh vực tài chính, nó được sử dụng để xác định hiệu suất trong tương lai của một cổ phiếu.
* Trong chăm sóc sức khỏe, nó được sử dụng để xác định sự kết hợp chính xác của các thành phần trong y học và nó cũng được sử dụng để phân tích bệnh sử của bệnh nhân để xác định bệnh tật.
* Trong thương mại điện tử, Random Forest được sử dụng để 
xác định xem khách hàng có thích sản phẩm hay không.
* Tuy nhiên, Random forests chậm tạo dự đoán bởi vì nó có nhiều cây quyết định. 
* Bất cứ khi nào nó đưa ra dự đoán, tất cả các cây trong rừng phải đưa ra dự đoán cho cùng một đầu vào cho trước và sau đó thực hiện bỏ phiếu trên đó. Toàn bộ quá trình này tốn thời gian.
* Vì vậy, tối ưu hóa bằng song song là rất cần thiết đối với mô hình Random Forest

# Tìm hiểu data
## Dữ liệu đầu vào đưa vào mô hình

* Link Dataset http://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)
* Tệp "sonar.mines" chứa 111 mẫu thu được bằng cách bật tín hiệu sonar ra khỏi một hình trụ kim loại ở nhiều góc độ khác nhau và trong các điều kiện khác nhau. 
* Tệp "sonar.rocks" chứa 97 mẫu thu được từ đá trong các điều kiện tương tự. Tín hiệu sonar truyền đi là một tiếng kêu được điều chỉnh tần số, tăng tần số. 
* Tập dữ liệu chứa các tín hiệu thu được từ nhiều góc độ khía cạnh khác nhau, trải dài 90 độ đối với hình trụ và 180 độ đối với đá.
* Mỗi mẫu là một tập hợp gồm 60 số trong phạm vi từ 0,0 đến 1,0. Mỗi số đại diện cho năng lượng trong một dải tần số cụ thể, được tích hợp trong một khoảng thời gian nhất định. 
* Khẩu độ tích hợp cho các tần số cao hơn xảy ra muộn hơn trong thời gian, vì các tần số này được truyền sau đó trong tiếng kêu.
* Nhãn liên kết với mỗi bản ghi chứa chữ cái "R" nếu đối tượng là một tảng đá và "M" nếu đó là mỏ (hình trụ kim loại).
![](https://i.imgur.com/zaH8WGi.png)

##Source
* Bộ dữ liệu được đóng góp vào việc thu thập điểm chuẩn bởi Terry Sejnowski, hiện đang làm việc tại Viện Salk và Đại học California tại San Deigo. 
* Bộ dữ liệu được phát triển với sự hợp tác của R. Paul Gorman của Trung tâm Công nghệ Hàng không Vũ trụ Allied-Signal.


In [ ]:
data = pd.read_csv('/content/sonar.all-data.csv')
dataset = data.values
data

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
203,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
204,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
205,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


#Tổng quan về thuật toán Random Forest
* Thuật toán Random Forest sẽ xây dựng nhiều cây quyết định bằng thuật toán Decision Tree
* Tuy nhiên mỗi cây quyết định sẽ khác nhau (có yếu tố random). Sau đó kết quả dự đoán được tổng hợp từ các cây quyết định
* Decision Tree là tên đại diện cho một nhóm thuật toán phát triển dựa trên Cây quyết định. Ở đó, mỗi Node của cây sẽ là các thuộc tính, và các nhánh là giá trị lựa chọn của thuộc tính đó. Bằng cách đi theo các giá trị thuộc tính trên cây,
Cây quyết định sẽ cho ta biết giá trị dự đoán.


##Ưu điểm

* Random Forest algorithm có thể sử dụng cho cả bài toán Classification và Regression
* Khi Forest có nhiều cây hơn, chúng ta có thể tránh được việc Overfitting với tập dữ liệu

## Cách hoạt động
* Chọn các mẫu ngẫu nhiên từ tập dữ liệu đã cho.
* Thiết lập cây quyết định cho từng mẫu và nhận kết quả dự đoán từ mỗi quyết định cây.
* Hãy bỏ phiếu cho mỗi kết quả dự đoán.
* Chọn kết quả được dự đoán nhiều nhất là dự đoán cuối cùng.
![](https://i.imgur.com/O5TQ5K6.png)


## Mã giả tạo ra nhiều cây 
* Chọn ngẫu nhiên “k” features từ tập “m” features. Chọn ngẫu nhiên số dòng dữ liệu từ dataset.
Để ý k << m

* Từ tập “k” features, tính toán ra node “d” là tốt nhất cho Node phân loại.

* Chia các node con theo node tốt nhất vừa tìm được

* Dùng thuật toán Decision Tree tạo cây cho Node tốt nhất.

* Lặp lại bước 1-4 để tạo ra “n” cây

#Code tuần tự

## Hàm load dữ liệu

In [ ]:
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:       
		csv_reader = reader(file)      
		for row in csv_reader:
			if not row:
				continue         
			dataset.append(row)
	return dataset

## Hàm chuyển đổi những dòng string thành float

In [ ]:
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

## Hàm chuyển đổi cột lable thành int (0 hoặc 1)

In [ ]:
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


## Hàm tách left,right dựa trên attribute và attribute value

In [ ]:
def test_split(index, value, dataset):
	left, right = np.array([dataset[0]]), np.array([dataset[0]])
	for row in dataset:
		if row[index] < value:
			left = np.append(left,[row],axis=0)
		else: 
			right = np.append(right,[row],axis=0)
	return left, right

##Hàm chia nhỏ data thành các folds để test

In [ ]:
def cross_validation_split(dataset, n_folds):
	dataset_split = np.array([dataset[0]])
	dataset_copy = np.array(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = np.array([dataset_copy[0]])
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold = np.append(fold,[dataset_copy[index]],axis=0)
			dataset_copy = np.delete(dataset_copy,index,axis=0)
		dataset_split = np.append(dataset_split,fold,axis = 0)
	dataset_split = np.delete(dataset_split,0,axis=0)
	newDataSet = np.array([dataset_split[0:fold_size]])
	for i in range(1,n_folds):
		newDataSet = np.append(newDataSet,[dataset_split[fold_size * i: fold_size * (i+1)]],axis=0)
	return newDataSet

##Hàm tính toán accuracy

In [ ]:
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

## Hàm tạo cây

In [ ]:
def build_tree(train, max_depth, min_size, n_features):
    #Building the tree involves creating the root node and 
	root = get_split(train, n_features)
    #calling the split() function that then calls itself recursively to build out the whole tree.
	split(root, max_depth, min_size, n_features, 1)
	return root

## Hàm get_split chọn điểm node tốt nhất của data

In [ ]:
def get_split(dataset, n_features):
	start2 = time.time()
	class_values = np.array([0.0,1.0])
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = np.array(0)
	i = 0
	while features.size < n_features + 1:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features = np.append(features, index)
	features = np.delete(features,0, axis=0)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
		end= time.time()
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

## Hàm split tạo cây khi có node root

In [ ]:
def split(node, max_depth, min_size, n_features, depth):
    #Firstly, the two groups of data split by the node are extracted for use and 
    #deleted from the node. As we work on these groups the node no longer requires access to these data.
	left, right = node['groups']
	del(node['groups'])
    
    #Next, we check if either left or right group of rows is empty and if so we create 
    #a terminal node using what records we do have.
	# check for a no split
	if not left.any() or not right.any():
		node['left'] = node['right'] = to_terminal(left + right)
		return
    #We then check if we have reached our maximum depth and if so we create a terminal node.
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
    #We then process the left child, creating a terminal node if the group of rows is too small, 
    #otherwise creating and adding the left node in a depth first fashion until the bottom of 
    #the tree is reached on this branch.
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
    #The right side is then processed in the same manner, 
    #as we rise back up the constructed tree to the root.
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

## Tạo các random subsample từ dataset (các dòng dữ liệu có thể bị trùng)

In [ ]:
def subsample(dataset, ratio):
	sample = np.array([dataset[0]])
	n_sample = round(len(dataset) * ratio)
	i = 0
	while i < n_sample:
		index = randrange(len(dataset))
		sample = np.append(sample,[dataset[index]],axis=0)
		i+=1 
	sample = np.delete(sample,0, axis=0)
	return sample

## Hàm tính gini index

In [ ]:
def gini_index(groups, class_values):
	gini = 0.0
    #for each class
	for class_value in class_values:
        #a random subset of that class
		for group in groups:
			size = len(group)
			if size == 0:
				continue
            #average of all class values
			proportion = [row[-1] for row in group].count(class_value) / float(size)
            #  sum all (p * 1-p) values, this is gini index
			gini += (proportion * (1.0 - proportion))
	return gini

## Hàm tạo terminal node

In [ ]:
# Create a terminal node value
def to_terminal(group):
    #select a class value for a group of rows. 
	outcomes = [row[-1] for row in group]
    #returns the most common output value in a list of rows.
	return max(set(outcomes), key=outcomes.count)

In [ ]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    #folds are the subsamples used to train and validate model
	folds = cross_validation_split(dataset, n_folds)
	scores = np.array([])
	i = 0
    #for each subsample
	for fold in folds:
        #create a copy of the data
		train_set = np.copy(folds)
        #remove the given subsample
		train_set = np.delete(train_set, i, axis=0)
		
		train_data = np.copy([train_set[0][0]])
		for item in train_set:
			for it in item:
				train_data = np.append(train_data,[it],axis=0)
        #init a test set
		train_data = np.delete(train_data, 0, axis=0)
		test_set = np.copy([fold[0]])
		newActual = np.array(fold)
        #add each row in a given subsample to the test set
		for row in newActual:
			row_copy = row
			row_copy[-1] = None
			test_set = np.append(test_set,[row_copy],axis=0)	
        #get predicted labls
		test_set = np.delete(test_set, 0 , axis=0)
		predicted = algorithm(train_data, test_set, *args)
        #get actual labels
		actual = [row[-1] for row in fold]
        #compare accuracy
		accuracy = accuracy_metric(actual, predicted)
        #add it to scores list, for each fold
		scores = np.append(scores,accuracy)
		i = i + 1
    #return all accuracy scores
	return scores
 
# Make a prediction with a decision tree
def predict(node, row):
    #Making predictions with a decision tree involves navigating the 
    #tree with the specifically provided row of data.
    #Again, we can implement this using a recursive function, where the same prediction routine is 
    #called again with the left or the right child nodes, depending on how the split affects the provided data.
    #We must check if a child node is either a terminal value to be returned as the prediction
    #, or if it is a dictionary node containing another level of the tree to be considered.
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']
 

# Test the random forest algorithm
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count,default=0)
 


## Hàm Random Forset tạo nhiều cây

In [ ]:
@jit
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	startRF = time.time()
	trees = np.array([])
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees = np.append(trees,tree)
	predictions = [bagging_predict(trees, row) for row in test]
	endRF = time.time()
	return(predictions)


In [ ]:
seed(1)
# load and prepare datad
filename = 'sonar.all-data.csv'
dataset = load_csv("/content/sonar.all-data.csv")
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 10
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
start = time.time()
for n_trees in [3]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))  
end = time.time()


<ipython-input-20-812213272fe6>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "random_forest" failed type inference due to: Untyped global name 'subsample': Cannot determine Numba type of <class 'function'>

File "<ipython-input-20-812213272fe6>", line 6:
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    <source elided>
	for i in range(n_trees):
		sample = subsample(train, sample_size)
  ^

  @jit
<ipython-input-20-812213272fe6>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "random_forest" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-20-812213272fe6>", line 5:
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    <source elided>
	trees = np.array([])
	for i in range(n_trees):
 ^

  @jit
/usr/local/lib/python3.7/dis

Trees: 3
Scores: [40. 60. 50. 30. 50. 45. 30. 50. 80. 60.]
Mean Accuracy: 49.500%


In [ ]:
time = end - start
results.loc['Tổng thời gian Code Tuần Tự', 'time (s)'] = time
results

,time (s)
Tổng thời gian Code Tuần Tự,357.123594


##Tính đúng đắn Code Tuần Tự

In [ ]:
df = pd.read_csv("sonar.all-data.csv" )
df["R"].value_counts()
repl = {"R": {"M": 1, "R": 0 }}
df_new = df.replace(repl)
df1 = df_new.drop(df.columns[len(df.columns)-1], axis=1)

In [ ]:
X = df1
print(X)
Y = df_new["R"].tolist()
print(Y)
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2, random_state=63)

     0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109  \
0    0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
1    0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
2    0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
3    0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
4    0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
202  0.0187  0.0346  0.0168  0.0177  0.0393  0.1630  0.2028  0.1694  0.2328   
203  0.0323  0.0101  0.0298  0.0564  0.0760  0.0958  0.0990  0.1018  0.1030   
204  0.0522  0.0437  0.0180  0.0292  0.0351  0.1171  0.1257  0.1178  0.1258   
205  0.0303  0.0353  0.0490  0.0608  0.0167  0.1354  0.1465  0.1123  0.1945   
206  0.0260  0.0363  0.0136  0.0272  0.0214  0.0338  0.0655  0.1400  0.1843   

     0.2111  ...  0.0232  0.0027  0.0065  0.0159  0

In [ ]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)
accuracy = []
clf1 = RandomForestClassifier(n_estimators=5,max_depth=10)
scores = cross_val_score(clf1, X, Y, cv=kf)
avg_score = np.mean(scores)
print(scores)
print(avg_score)

[0.19047619 0.66666667 0.71428571 0.61904762 0.42857143 0.33333333
 0.57142857 0.5        0.8        0.8       ]
0.5623809523809523


## Nhận xét:
* So với kết quả với thư viện RandomForestClassifier tính đúng đắn ổn.

##Thiết kế:
* Build Grow(tree):
* Đối với tất cả dữ liệu trong node này, hãy tìm điểm node tốt nhất ( Parallel)
* Tạo con bên trái và con bên phải bằng cách chia nhỏ dữ liệu tại điểm phân tách (Parallel)
* Nếu left child không phải terminal, grow left child
* Nếu right child không phải terminal, grow right child

#Nguồn tham khảo code tuần tự:
https://github.com/llSourcell/random_forests